In [1]:
import pandas as pd
import numpy as np
import json
import re
import glob
from datetime import date

import sys
sys.path.insert(0, '/Users/matty_gee/Desktop/SNT/SNT-behavior/Online/Prolific/Code/0_utilities')
# from prolific_behavior_general_utils import *
from prolific_generic_utils import *

# info about the validated decisions: e.g., the continuous ratings etc..
decisions_info = pd.read_excel('/Users/matty_gee/Desktop/SNT/SNT-Behavior/Online/Prolific/Info/TaskValidation_Val20_SortedOptionMeans_Mem50_n81.xlsx')
decisions_info = decisions_info.sort_values(by = 'Decision').reset_index(drop=True)
char_roles = ['first', 'second', 'powerful', 'boss', 'assistant', 'neutral'] # order in js...
char_role = ['first', 'second', 'powerful', 'boss', 'assistant', 'neutral'] 

//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


# From VT server

In [2]:
data_dir       = '/Users/matty_gee/Desktop/SNT/SNT-behavior/Online/Prolific/Data/Original_2021/Task'
sub_files      = [sub_data for sub_data in glob.glob(data_dir + "/Raw-VT_server/*.json.txt")]
sub_ids        = []
dates          = []
posttask_dfs   = []
browser_list   = []
task_responses = []
task_versions  = []
memory_dfs     = []


for file in sub_files:
    with open(file) as f:
        
        # get data
        data = json.load(f)
        if ('preview=1' not in data['metadata']['url']) and ('prolific_pid' in data['metadata']['social_task_data']): # exclude previews/tests
            
            exp_data = data['metadata']['social_task_data']
            sub_id = exp_data['prolific_pid']
            sub_ids.append(sub_id)
            
            date = sub_files[0].split('/')[-1].split('.')[2].split('T')[0]
            dates.append(date[0:4] + '-' + date[4:6] + '-' + date[6:])

            # check for browser type - dots debugging hppefully....
            if 'browser' in data['metadata']:
                browser = data['metadata']['browser']
            else:
                browser = 'unknown'
            browser_list.append((exp_data['prolific_pid'], exp_data['session_id'], browser))

            ############################################################
            # task version, character info
            ############################################################

            narrative_ver = exp_data['version']
            task_versions.append(narrative_ver)
            
            char_name     = [name for name, img in exp_data['character_imgs'].items()]
            char_img      = [img for name, img in exp_data['character_imgs'].items()]

            # if the first character is a woman have to reorganize...
            female_first = [1,0,2,3,5,4]
            if 'F' in narrative_ver:
                char_name = [char_name[f] for f in female_first]
                char_img = [char_img[f] for f in female_first]

            char_gender    = []
            char_skincolor = []
            for img in char_img: 
                if 'Br_' in img:
                    char_skincolor.append('brown')
                else:
                    char_skincolor.append('white')
                if 'Male' in img:
                    char_gender.append('man')
                else:
                    char_gender.append('woman')

            ############################################################
            # narrative task
            ############################################################

            # get task responses, options order
            narrative_resp = exp_data['narrative_resps'][1:-1].split(',')
            narrative_resp = list(map(int, narrative_resp)) # turn into integers
            narrative_opt_order = exp_data['narrative_opts_order'][1:-1].split('],[')

            # standardize & collect responses, depending on alphabetical ordering of options
            narrative_resp_data = np.zeros(len(narrative_resp))  
            affl_cont = []
            pwr_cont  = []
            affl_disc = []
            pwr_disc  = []
            for q, question in enumerate(narrative_opt_order):

                # get response, accounting for option randomization 
                opt1 = remove_nontext(question.split('","')[0])
                opt2 = remove_nontext(question.split('","')[1])
                sort_ix = np.argsort((opt1,opt2)) # order options alphabetically
                resp = sort_ix[narrative_resp[q]-1]+1 # choice -> 1 or 2, depending on alphabetical ordering
                narrative_resp_data[q] = resp

                # turn response into the validated ratings
                decision = decisions_info.iloc[q]
                affl_cont.append(decision['Opt{}_Affl_Mean'.format(int(resp))])
                pwr_cont.append(decision['Opt{}_Pwr_Mean'.format(int(resp))])
                affl_disc.append(decision['Opt{}_Affl_Discrete'.format(int(resp))])
                pwr_disc.append(decision['Opt{}_Pwr_Discrete'.format(int(resp))])

            # all_responses
            task_responses.append(np.array(pwr_disc) + np.array(affl_disc))
            
            # get rts 
            narrative_rts = np.array([int(i, base=16) for i in exp_data['narrative_rts'][1:-1].split(',')])
            decision_rts  = narrative_rts[np.array(decisions_info['Slide_num'])-1]

            ############################################################
            # create an output file (will compute geometry in matlab)
            ############################################################

            narrative_df = pd.DataFrame((narrative_resp, affl_disc, pwr_disc, decision_rts)).T
            narrative_df.columns = ['button_press','affil', 'power', 'rt']
            narrative_df.insert(0, 'trial_num', np.arange(1,64))
            narrative_df.to_excel(data_dir + '/Organized/SNT_' + sub_id + '.xlsx', index=False)

            ############################################################
            # memory task
            ############################################################

            # sort question/response alphabetically
            mem_resp = [remove_nontext(x) for x in exp_data['memory_resps'][1:-1].split(',')]
            mem_ques = [remove_nontext(x) for x in exp_data['memory_quests_order'][1:-1].split(',')]
            mem_data = sorted(list(zip(mem_ques, mem_resp)))

            # evaluate responses for which role it corresponds to and if that is correct 
            # 1 = first, 2 = second, 3 = newcomb, 4 = hayworth, 5 = assistant, 6 = neutral
            mem_correct = [1,3,5,3,5,0,0,0,3,1,2,2,1,3,5,2,1,0,4,5,5,4,4,4,2,2,0,1,4,3] # according to the order of characters in char_role
            mem_questions = ['mem' + str(q+1) + '_'  + char_role[r] for q,r in enumerate(mem_correct)]
            mem_summary = np.zeros(6) # summary % by character...
            mem_responses = []
            for r,resp in enumerate(mem_data): 
                mem_responses.append(char_role[char_name.index(resp[1])]) # find the role of the character response
                if resp[1] == char_name[mem_correct[r]]: mem_summary[mem_correct[r]]+=1 # if correct 
            memory_df = pd.DataFrame(mem_summary/5, columns=['memory'])
            memory_dfs.append(pd.DataFrame(np.array(mem_responses).reshape(1,-1), columns=mem_questions)) 

            ############################################################
            # dots task 
            ############################################################

            characters_split = exp_data['dots_resps'].split('],[')
            dots_data = np.zeros((6,3)) # affil, pwr, duration

            for c, character in enumerate(characters_split):

                character_data = character.split(',') 
                character_name = remove_nontext(character_data[0].split(':')[0])
                character_img  = character_data[0].split(':')[1].split('.png')[0]

                # can simplify this below:
                role_id = char_img.index(character_img) 
                dots_data[role_id,0] = character_data[1].split(':')[1] # affiliation
                dots_data[role_id,1] = character_data[2].split(':')[1] # power
                dots_data[role_id,2] = remove_nontext(character_data[3].split(':')[1]) # duration

            dots_df = pd.DataFrame(dots_data, columns=['dots_affil', 'dots_power', 'duration(ms)'])

            ############################################################
            # perception tasks
            ############################################################

            perception_tasks = ['liking','competence','similarity']
            perception_resps = np.zeros((6,len(perception_tasks)))
            perception_rts   = np.zeros((6,len(perception_tasks)))
            for t,task in enumerate(perception_tasks):
                perception_resps[:,t] = np.array(list(map(int, exp_data[task + '_resps'][1:-1].split(','))))
                perception_rts[:,t]   = list(map(int, exp_data[task + '_rts'][1:-1].split(',')))

            ############################################################
            # emotion tasks
            ############################################################

            emot_dims = ['"arousal"','"valence"','"stability"']
            ratings = np.zeros((6,len(emot_dims)))

            # will be same character order as other perception tasks
            for r, resps in enumerate(exp_data['emotion_resps'].split('],')):
                resps    = resps.replace('[', "").replace(']', "") # annoying way to do this...
                dims     = [r for r in resps.split(',')[::2]] # get the order of the dims in task
                dims_ixs = np.array([dims.index(dim) for dim in emot_dims]) # turn into ixs, based on my order
                ratings[r,:]  = np.array([int(r) for r in resps.split(',')[1::2]])[dims_ixs] # get & order ratings

            # gather + sort by the role order (based on image order) (character X task)
            perception_resps = np.concatenate((perception_resps, ratings),axis=1)
            role_sorting     = np.array([exp_data['perception_character_order'].index(char) for char in char_name])
            perception_data  = perception_resps[role_sorting]
            perception_df    = pd.DataFrame(perception_data, columns=['liking','competence','similarity','arousal','valence','stability'])

            ############################################################
            # output perception 
            ############################################################        

            # per subject
            perc_mem_df = pd.concat([memory_df, dots_df, perception_df], axis=1)
            perc_mem_df.insert(0, 'task_version', narrative_ver)
            perc_mem_df.insert(1, 'name', np.array(char_name))
            perc_mem_df.insert(2, 'gender', np.array(char_gender))
            perc_mem_df.insert(3, 'skincolor', np.array(char_skincolor))
            perc_mem_df.insert(4, 'image', char_img)
#             perc_mem_df.to_excel(data_dir + '/Organized/' + sub_id + '_posttask_data.xlsx', index=False)

            # for summary sheet
            perc_mem_flattened = []
            perc_mem_cols = []
            for r,row in enumerate(perc_mem_df.iterrows()):
                perc_mem_flattened.extend([row[1][col] for col in perc_mem_df.columns])
                perc_mem_cols.extend([char_role[r]+'_'+col for col in perc_mem_df.columns])
                
            perc_mem_flattened_df = pd.DataFrame(np.array(perc_mem_flattened).reshape(1,-1), columns=perc_mem_cols)

            ############################################################
            # iq
            ############################################################       

            iq_ques = [remove_nontext(ques) for ques in exp_data['iq_quests'][1:-1].split(',')]
            iq_resp = [remove_nontext(resp) for resp in exp_data['iq_resps'][1:-1].split(',')]
            iq_rt   = exp_data['iq_rts'][1:-1].split(',')
            answers =  [["VR4",5],["VR16","Its"],["VR17",47],["VR19","Sunday"],
                        ["LN7","X"],["LN33","G"],["LN34","X"],["LN58","N"],
                        ["MX45","E"],["MX46","B"],["MX47","B"],["MX55","D"],
                        ["R3D3","C"],["R3D4","B"],["R3D6","F"],["R3D8","G"]]
            iq_correct = np.zeros(len(answers))
            for answer in answers: 
                for (q,ques), resp in zip(enumerate(iq_ques), iq_resp):
                    if ques == answer[0]:
                        if resp == answer[1]:
                            iq_correct[q] = 1

            ############################################################
            # end Qs
            ############################################################ 

            end_questions = exp_data['end_questions'].split(';') #re.findall(r':(\w+)', exp_data['end_questions']) # this excludes free resp for now
            good = [resp for resp in end_questions if 'good:' in resp][0].split(':')[1]
            bad = [resp for resp in end_questions if 'bad:' in resp][0].split(':')[1]
            engagement = [resp for resp in end_questions if 'engagement:' in resp][0].split(':')[1]
            difficulty = [resp for resp in end_questions if 'difficulty:' in resp][0].split(':')[1]
            relatability = [resp for resp in end_questions if 'relatability:' in resp][0].split(':')[1]
            end_resps  = np.array([good, bad, int(engagement), int(difficulty), int(remove_nontext(relatability))])

            ############################################################
            # organize all post-task
            ############################################################

            colnames = np.concatenate((perc_mem_cols, ['mean_memory','mean_liking','mean_competence','mean_similarity','mean_arousal','mean_valence','mean_stability'], np.array(['iq_' + a[0] for a in answers]), ['iq_score', 'end_good','end_bad','end_engagement','end_difficulty','end_relatability']))
            sub_data = np.concatenate((perc_mem_flattened, np.array([np.mean(mem_summary/5)]), np.mean(perception_data,axis=1), iq_correct, np.array([np.mean(iq_correct)]), end_resps)).reshape(1,-1)
            posttask_dfs.append(pd.DataFrame(sub_data, columns = colnames))


out_dir = '/Users/matty_gee/Desktop/SNT/SNT-behavior/Online/Prolific/Data/Original_2021/'

# ############################################################
# # task decisions
# ############################################################ 

# task_responses_df = pd.DataFrame(task_responses, columns=['decision_' + '{:02d}'.format(n) for n in np.arange(1,64)])
# task_responses_df.insert(0, 'sub_id', np.array(sub_ids))
# task_responses_df.insert(1, 'task_version', task_versions)
# task_responses_df.to_excel(out_dir + '/Prolific_task_responses_n' + str(len(sub_ids)) + '.xlsx', index=False)

############################################################
# posttask summary
############################################################    

posttask_df_summary = pd.concat([pd.concat(posttask_dfs), pd.concat(memory_dfs)], axis=1)
posttask_df_summary.insert(0, 'sub_id', np.array(sub_ids))
posttask_df_summary.insert(1, 'task_version', task_versions)
posttask_df_summary.insert(2, 'date', dates)
posttask_df_summary.to_excel(out_dir + '/Posttask-original-VT_summary_n' + str(len(sub_ids)) + '.xlsx', index=False)

# pd.DataFrame(browser_list, columns = ['sub_id', 'sess_id', 'browser']).to_excel(out_dir + '/Prolific_completed_list_n' + str(len(sub_ids)) + '.xlsx', index=False)


# From pavlovia

In [3]:
# 1 = first, 2 = second, 3 = newcomb, 4 = hayworth, 5 = assistant, 6 = neutral
def get_memory(data, char_names):
    
    try:
        memory_data = data['memory_resps'].values[0]
    except: 
        memory_data = data['character_memory'].values[0]

    memory = [ques.split(';')[1:2] for ques in memory_data.split('","')]
    mem_ques = [mem[0].split(':')[0] for mem in memory]
    mem_resp = [mem[0].split(':')[1] for mem in memory]

    mem_data = sorted(list(zip(mem_ques, mem_resp)))

    # 1 = first, 2 = second, 3 = newcomb, 4 = hayworth, 5 = assistant, 6 = neutral
    mem_correct = [1,3,5,3,5,0,0,0,3,1,2,2,1,3,5,2,1,0,4,5,5,4,4,4,2,2,0,1,4,3]# according to the order of characters in char_role
    mem_acc = np.zeros(6) # organize by character...
    for r,resp in enumerate(mem_data): 
        if resp[1] == char_names[mem_correct[r]]:
            mem_acc[mem_correct[r]]+=1
    mem_acc_df = pd.DataFrame((mem_acc/5).reshape(1,-1), columns=[char + '_memory' for char in char_roles])
    mem_acc_df.insert(6, 'mean_memory', np.mean(mem_acc_df.values))

    # also output the response
    mem_questions = ['mem' + str(q+1) + '_'  + char_roles[r] for q,r in enumerate(mem_correct)]
    mem_responses = [char_roles[char_names.index(resp[1])] for r,resp in enumerate(mem_data)] # find the role of the character response
    mem_resp_df = pd.DataFrame(np.array(mem_responses).reshape(1,-1), columns=mem_questions)

    return pd.concat([mem_acc_df, mem_resp_df], axis=1) 

# dumb to have different orders everywhere...should just be explicit...
img_sets = {'OFA': ['OlderFemaleBl_2','OlderMaleW_1','OlderMaleW_4','OlderFemaleBr_3','OlderMaleBr_2','OlderFemaleW_1'],
            'OFB': ['OlderFemaleW_2','OlderMaleBr_1','OlderMaleBl_3','OlderFemaleW_3','OlderMaleW_5','OlderFemaleBl_1'],
            'OFC': ['OlderFemaleBl_2','OlderMaleBr_1','OlderMaleW_5','OlderFemaleW_3','OlderMaleBr_4','OlderFemaleW_1'],
            'OFD': ['OlderFemaleW_2','OlderMaleW_1','OlderMaleBr_3','OlderFemaleBr_3','OlderMaleW_5','OlderFemaleBl_1'],
            'OMA': ['OlderMaleBr_2','OlderFemaleW_2','OlderFemaleW_3','OlderMaleBr_1','OlderFemaleBr_5','OlderMaleW_5'],
            'OMB': ['OlderMaleW_1','OlderFemaleBl_2','OlderFemaleBl_3','OlderMaleW_4','OlderFemaleW_1','OlderMaleBr_4'], 
            'OMC': ['OlderMaleBr_4','OlderFemaleBl_2','OlderFemaleW_3','OlderMaleW_3','OlderFemaleBl_1','OlderMaleW_5'],
            'OMD': ['OlderMaleW_1','OlderFemaleW_2','OlderFemaleBr_5','OlderMaleBr_3','OlderFemaleW_1','OlderMaleBr_4'],
            'YFA': ['YoungerFemaleBr_1','YoungerMaleW_4','YoungerMaleW_3','OlderFemaleBr_5','OlderMaleBr_4','OlderFemaleW_1'],
            'YFB': ['YoungerFemaleW_3','YoungerMaleBr_2','OlderMaleBr_3','OlderFemaleW_4','YoungerMaleW_2','OlderFemaleBl_1'],
            'YFC': ['YoungerFemaleBr_1','YoungerMaleBr_2','OlderMaleW_4','OlderFemaleW_3','OlderMaleBr_4','OlderFemaleW_1'],
            'YFD': ['YoungerFemaleW_3','YoungerMaleW_4','OlderMaleBr_3','OlderFemaleBr_5','OlderMaleW_5','OlderFemaleBl_1'],
            'YMA': ['YoungerMaleBr_2','YoungerFemaleW_3','OlderFemaleW_4','OlderMaleBr_3','OlderFemaleBl_1','YoungerMaleW_2'],
            'YMB': ['YoungerMaleW_4','YoungerFemaleBr_1','OlderFemaleBr_5','YoungerMaleW_3','OlderFemaleW_1','OlderMaleBr_4'],
            'YMC': ['YoungerMaleBr_2','YoungerFemaleBr_1','OlderFemaleW_3','OlderMaleW_3','OlderFemaleBl_1','OlderMaleW_5'],
            'YMD': ['YoungerMaleW_4','YoungerFemaleW_3','OlderFemaleBr_3','OlderMaleBr_3','OlderFemaleW_1','OlderMaleBr_4']}

In [4]:
data_dir = '/Users/matty_gee/Desktop/SNT/SNT-behavior/Online/Prolific/Data/Original_2021'
sub_files = [sub_data for sub_data in glob.glob(data_dir + "/Task/Raw-Pavlovia/*.csv")]
task_vers = []
dates     = []
posttask_dfs = []
for file in sub_files:
    
    data = pd.read_csv(file)
    sub_id = data['prolific_id'].values[0]
    if sub_id not in ['5fb28e64134ec0894396633a', '60393e9ddde61e3a5463746d']:

        #################################################################
        # task version info
        #################################################################
        
        task_ver = data['task_ver'].values[0]
        task_vers.append(task_ver)
        
        date = file.split('/')[-1].split('_')[3]
        dates.append(date)
        
        # character names  
        char_imgs = img_sets[task_ver]
        if 'Male' in char_imgs[0]:
            char_names = ['Chris','Maya','Newcomb','Hayworth','Kayce','Anthony']
        else:
            char_names = ['Maya','Chris','Newcomb','Hayworth','Anthony','Kayce']
        
        # character demographics                    
        char_imgs
        char_gender    = []
        char_skincolor = []
        for img in char_imgs: 
            if 'Br_' in img:
                char_skincolor.append('brown')
            else:
                char_skincolor.append('white')
            if 'Male' in img:
                char_gender.append('man')
            else:
                char_gender.append('woman')
        gender_df = pd.DataFrame(np.array(char_gender).reshape(1,-1), index=[sub_id], columns=[role + '_gender' for role in char_roles])
        skincolor_df = pd.DataFrame(np.array(char_skincolor).reshape(1,-1), index=[sub_id], columns=[role + '_skincolor' for role in char_roles])

        #################################################################
        # narrative decisions
        #################################################################
        
        # narrative responses  
        try: # changed the name at some point
            narrative_resp = np.array([int(re.sub('["\]"]', '', d.split(':')[1])) for d in data['narrative_choices'].values[0].split(',')])
        except:
            narrative_resp = np.array([int(re.sub('["\]"]', '', d.split(':')[1])) for d in data['narrative_resps'].values[0].split(',')])

        narrative_opt_order = data['narrative_opts_order'].values[0].split('","')
        narrative_resp_data = np.zeros(len(narrative_resp))
        affl = []
        pwr  = []
        for q, question in enumerate(narrative_opt_order):
            # reallllyyyy did this dumb...
            question = ''.join(filter(lambda item: not item.isdigit(), question))
            if ';' in question:
                opt1 = remove_nontext(question.split(';')[1])
                opt2 = remove_nontext(question.split(';')[2])
            elif ',"' in question:
                opt1 = remove_nontext(question.split(',"')[0])
                opt2 = remove_nontext(question.split(',"')[1]) 
            elif '.,' in question:
                opt1 = remove_nontext(question.split('.,')[0])
                opt2 = remove_nontext(question.split('.,')[1]) 
                
            sort_ix = np.argsort((opt1,opt2)) # order options alphabetically
            resp = sort_ix[narrative_resp[q]-1]+1 # choice -> 1 or 2, depending on alphabetical ordering
            narrative_resp_data[q] = resp
            decision = decisions_info.iloc[q]
            affl.append(decision['Opt{}_Affl_Discrete'.format(int(resp))])
            pwr.append(decision['Opt{}_Pwr_Discrete'.format(int(resp))])
        narrative_rts = np.array([int(remove_nontext(rt)) for rt in data['narrative_rts'].values[0].split(',')])
        decision_rts  = narrative_rts[np.array(decisions_info['Slide_num'])-1]
        narrative_df = pd.DataFrame((narrative_resp, affl, pwr, decision_rts)).T
        narrative_df.columns = ['button_press', 'affil', 'power', 'rt']
        narrative_df.insert(0, 'trial_num', np.arange(1,64))
        narrative_df.to_excel(data_dir + '/Task/Organized/SNT_' + sub_id + '.xlsx', index=False)
        
        ######################################################################
        # POST TASK 
        ######################################################################
        
        # memory
        memory_df = get_memory(data, char_names)
        memory_df.index = [sub_id]
        
        # dots 
        dots_df = pd.DataFrame(index=[sub_id], columns=[char + '_dots_' + dim for char in char_roles for dim in ['affil','power']])
        for row in data['dots_resps'].values[0].split(','):
            row_split = row.split(';')
            role = char_roles[char_names.index(remove_nontext(row_split[0].split(':')[0]))]
            dots_df[role + '_dots_affil'] = float(row_split[1].split(':')[1])
            dots_df[role + '_dots_power'] = float(row_split[2].split(':')[1])

        # relationship evaluations
        try:
            character_relationship = [char.split(';') for char in data['character_relationship'].values[0].split(',')]
        except: 
            character_relationship = [char.split(';') for char in data['relationship_feelings'].values[0].split(',')]
            
        rel_df = pd.DataFrame(index=[sub_id], columns=[char + '_' + rel for char in char_roles for rel in ['arousal','valence','stability']])
        for char in character_relationship:
            role = char_roles[char_names.index(remove_nontext(char[0]))]
            for r in range(1,4):
                rel = char[r].split(':')[0]
                rating = float(char[r].split(':')[1])
                rel_df[role + '_' + rel] = rating
        for dim in ['arousal','valence','stability']:
            rel_df['mean_' + dim] = np.mean(rel_df[[char + '_' + dim for char in char_roles]].values)

        # character evaluations    
        dims = ['status', 'competence', 'youthful', 'gender', 'similarity', 'approachable', 'dominant', 'likability', 'trustworthy', 'race']
        try:
            resps = [row.split(';') for row in data['character_dimensions'].values[0].split(',')]
        except: 
            resps = [row.split(';') for row in data['character_ratings'].values[0].split(',')]
        
        dim_df = pd.DataFrame(index=[sub_id], columns=[char + '_' + dim for char in char_roles for dim in dims])
        for resp in resps:
            name = remove_nontext(resp[0])
            if name != 'attention':
                role = char_roles[char_names.index(remove_nontext(resp[0]))]
                dim_df[role + '_' + resp[1].split(':')[0]] = int(resp[1].split(':')[1])

        for dim in ['status', 'competence', 'youthful', 'gender', 'similarity', 'approachable', 'dominant', 'likability', 'trustworthy']:
            dim_df['mean_' + dim] = np.mean(dim_df[[char + '_' + dim for char in char_roles]].values)

        # add race
        try:
            char_race = data['character_race'].values[0].split('","')
        except:
            char_race = data['race_categorization'].values[0].split('","')
        for resp in char_race:
            role = char_roles[char_names.index(remove_nontext(resp.split(';')[0].split(',')[0]))]
            dim_df[role + '_race'] = resp.split(';')[1].split(':')[1]
            
        posttask_dfs.append(pd.concat([gender_df, skincolor_df, memory_df, dots_df, dim_df, rel_df], axis=1))
    
posttask_df = pd.concat(posttask_dfs)
posttask_df.index.name = 'sub_id'
posttask_df.reset_index(inplace=True)
posttask_df.insert(1, 'task_version', task_vers)
posttask_df.insert(2, 'date', dates)
posttask_df.to_excel(data_dir + '/Summary/Posttask-pavlovia_summary_n' + str(len(posttask_df)) + '.xlsx', index=False)
